In [ ]:
# install necessary packages
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("TCGAbiolinks")
install.packages("ranger")
install.packages('caret')
install.packages('DALEX')
install.packages('pROC')
install.packages('SummarizedExperiment')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.19 (BiocManager 1.30.25), R 4.4.1 (2024-06-14)

Installing package(s) 'BiocVersion', 'TCGAbiolinks'

also installing the dependencies ‘plogr’, ‘png’, ‘Biostrings’, ‘RSQLite’, ‘KEGGREST’, ‘filelock’, ‘UCSC.utils’, ‘GenomeInfoDbData’, ‘zlibbioc’, ‘matrixStats’, ‘abind’, ‘SparseArray’, ‘AnnotationDbi’, ‘BiocFileCache’, ‘BiocGenerics’, ‘GenomeInfoDb’, ‘XVector’, ‘R.oo’, ‘R.methodsS3’, ‘MatrixGenerics’, ‘Biobase’, ‘S4Arrays’, ‘DelayedArray’, ‘downloader’, ‘biomaRt’, ‘GenomicRanges’, ‘XML’, ‘plyr’, ‘IRanges’, ‘S4Vectors’, ‘R.utils’, ‘SummarizedExperiment’, ‘TCGAbiolinksGUI.data’


Old packages: 'askpass', 'commonmark', 'credentials', 'sys', 'xfun'

Installing package into ‘/usr/local/lib/R/site-lib

In [ ]:
library(caret)
library(TCGAbiolinks)
library(DALEX)
library(pROC)
library(ranger)
library(SummarizedExperiment)

Loading required package: ggplot2

Loading required package: lattice

Welcome to DALEX (version: 2.4.3).
Find examples and detailed introduction at: http://ema.drwhy.ai/
Additional features will be available after installation of: ggpubr.
Use 'install_dependencies()' to get all suggested dependencies

Type 'citation("pROC")' for a citation.


Attaching package: ‘pROC’


The following objects are masked from ‘package:stats’:

    cov, smooth, var


Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDi

In [ ]:
query_DGE <- GDCquery(project = "TCGA-LGG",
                  data.category = "Transcriptome Profiling",
                  experimental.strategy = "RNA-Seq",
                  workflow.type = "STAR - Counts",
                  access = "open")

--------------------------------------

o GDCquery: Searching in GDC database

--------------------------------------

Genome of reference: hg38

--------------------------------------------

oo Accessing GDC. This might take a while...

--------------------------------------------

ooo Project: TCGA-LGG

--------------------

oo Filtering results

--------------------

ooo By access

ooo By experimental.strategy

ooo By workflow.type

----------------

oo Checking data

----------------

ooo Checking if there are duplicated cases

ooo Checking if there are results for the query

-------------------

o Preparing output

-------------------



In [ ]:
#get results of the query
output_query_DGE_TCGA <- getResults(query_DGE)

In [ ]:
GDCdownload(query_DGE)


GDCdownload will download 534 files. A total of 2.267094301 GB

The total size of files is big. We will download files in chunks




Downloading: 250 MB     

Downloading: 250 MB     

Downloading: 67 MB     

In [ ]:
DGE_data <- GDCprepare(query_DGE)

|====================================================|100%                      Completed after 35 s 


Starting to add information to samples

 => Add clinical information to samples

 => Adding TCGA molecular information from marker papers

 => Information will have prefix 'paper_' 

lgg subtype information from:doi:10.1016/j.cell.2015.12.028

Available assays in SummarizedExperiment : 
  => unstranded
  => stranded_first
  => stranded_second
  => tpm_unstrand
  => fpkm_unstrand
  => fpkm_uq_unstrand



In [ ]:
DEA_matrix <- assay(DGE_data, "unstranded")

In [ ]:
dim(DEA_matrix)

[1] 60660   534

In [ ]:
# import DGE metadata
DGE_metadata <- read.csv("https://raw.githubusercontent.com/mahiiC/HackBio_Internship_2024/refs/heads/main/Stage_4%20Task/Data/DGE_metadata_with_IDHstatus.csv", row.names = 1)
head(DGE_metadata)

,patient,sample_names,sample_type,IDH.status,IDH.codel.subtype,IDH.specific.DNA.Methylation.Cluster,IDH.specific.RNA.Expression.Cluster
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,TCGA-CS-4938,TCGA-CS-4938-01B-11R-1896-07,Primary Tumor,Mutant,IDHmut-non-codel,IDHmut-K2,IDHmut-R3
2,TCGA-CS-4941,TCGA-CS-4941-01A-01R-1470-07,Primary Tumor,WT,IDHwt,IDHwt-K2,IDHwt-R2
3,TCGA-CS-4942,TCGA-CS-4942-01A-01R-1470-07,Primary Tumor,Mutant,IDHmut-non-codel,IDHmut-K2,IDHmut-R3
4,TCGA-CS-4943,TCGA-CS-4943-01A-01R-1470-07,Primary Tumor,Mutant,IDHmut-non-codel,IDHmut-K2,IDHmut-R3
5,TCGA-CS-4944,TCGA-CS-4944-01A-01R-1470-07,Primary Tumor,Mutant,IDHmut-non-codel,IDHmut-K2,IDHmut-R3
6,TCGA-CS-5390,TCGA-CS-5390-01A-02R-1470-07,Primary Tumor,Mutant,IDHmut-codel,IDHmut-K3,IDHmut-R1


In [ ]:
# Making sample_names as rownames of DGE_metadata
rownames(DGE_metadata) <- DGE_metadata[,2]
DGE_metadata <- DGE_metadata[,-2]

In [ ]:
# checking if all row names of DGE_metadata are present in column names DGE_norm_data
all(colnames(DEA_matrix) %in% rownames(DGE_metadata))

[1] TRUE

In [ ]:
# checking if all row names of DGE_metadata are in the same order as column names in DGE_norm_data
all(rownames(DGE_metadata) == colnames(DEA_matrix))

[1] FALSE

In [ ]:
# reordering dataset
DEA_matrix <- DEA_matrix[, rownames(DGE_metadata)]
all(rownames(DGE_metadata) == colnames(DEA_matrix))

[1] TRUE

In [ ]:
set.seed(12345)

### **Data Preprocessing for ML**

In [ ]:
# selecting the top 8000 variable genes
data_trans <- data.frame(t(DEA_matrix))
sds <- apply(data_trans, 2, sd) #standard deviation
top_preds <- order(sds, decreasing = T)[1:8000]
data_trans <- data_trans[, top_preds]
dim(data_trans)

[1]  534 8000

In [ ]:
head(data_trans)

,ENSG00000131095.13,ENSG00000210082.2,ENSG00000198804.2,ENSG00000198886.2,ENSG00000120885.22,ENSG00000113140.11,ENSG00000118271.12,ENSG00000198938.2,ENSG00000198712.1,ENSG00000198727.2,⋯,ENSG00000108306.13,ENSG00000160097.20,ENSG00000160211.19,ENSG00000145348.17,ENSG00000101417.12,ENSG00000140259.7,ENSG00000141385.10,ENSG00000101019.22,ENSG00000109618.12,ENSG00000183549.10
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
TCGA-CS-4938-01B-11R-1896-07,4116776,990695,1139736,1403693,136718,388758,2,350788,458698,450084,⋯,1880,931,1515,1612,1850,2484,2598,2128,661,620
TCGA-CS-4941-01A-01R-1470-07,1986497,434211,452217,350787,561733,197571,4,150503,210737,170910,⋯,1895,706,2182,1957,1825,2795,2928,3269,744,633
TCGA-CS-4942-01A-01R-1470-07,2337734,696161,512382,459020,296710,228339,2,227203,274289,266559,⋯,2062,1584,1739,2392,843,2966,2227,2338,740,909
TCGA-CS-4943-01A-01R-1470-07,654588,605628,398355,330243,168987,181947,0,180345,152017,144796,⋯,3270,711,1916,1979,255,3932,1868,2223,985,282
TCGA-CS-4944-01A-01R-1470-07,5644438,1034671,507632,538086,4624713,231326,11,234253,279080,268078,⋯,1345,1489,2068,1346,988,2599,1460,2170,393,1098
TCGA-CS-5390-01A-02R-1470-07,471764,1052898,951973,730924,191335,163448,1,311624,358225,218747,⋯,2984,801,1788,3302,2412,2541,2476,2764,908,284


In [ ]:
keep_genes = colSums(data_trans) >= 10
data_trans <- data_trans[, keep_genes]
data_trans <- data_trans[rowSums(data_trans) >= 5, ]

In [ ]:
dim(data_trans)

[1]  534 8000

In [ ]:
BiocManager::install('DESeq2')
library(DESeq2)

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.19 (BiocManager 1.30.25), R 4.4.1 (2024-06-14)

Installing package(s) 'DESeq2'

also installing the dependencies ‘formatR’, ‘lambda.r’, ‘futile.options’, ‘futile.logger’, ‘snow’, ‘BH’, ‘BiocParallel’, ‘locfit’, ‘RcppArmadillo’


Old packages: 'askpass', 'commonmark', 'credentials', 'sys', 'xfun'



In [ ]:
# Removing genes with Near Zero Variation (NZV)
data_zero <- preProcess(data_trans, method = "nzv", uniqueCut = 15)
data_trans <- predict(data_zero, data_trans)

In [ ]:
dim(data_trans)

[1]  534 8000

In [ ]:
# Convert data_trans to numeric matrix
data_trans <- as.matrix(data_trans)  # Convert to a matrix first
data_trans <- data_trans + 1 # Adding pseudocount to avoid genes with all zeros

# Normalization with DESeq2
dds <- DESeqDataSetFromMatrix(countData = data_trans, colData = DataFrame(row.names=colnames(data_trans)), design = ~ 1) # 1 passed to the function because of no model
dds <- DESeq(dds)

converting counts to integer mode

Warning message in DESeq(dds):
“the design is ~ 1 (just an intercept). is this intended?”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

-- note: fitType='parametric', but the dispersion trend was not well captured by the
   function: y = a/x + b, and a local regression fit was automatically substituted.
   specify fitType='local' or 'mean' to avoid this message next time.

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 401 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



In [ ]:
# vst transformation
vsd <- assay(varianceStabilizingTransformation(dds))

-- note: fitType='parametric', but the dispersion trend was not well captured by the
   function: y = a/x + b, and a local regression fit was automatically substituted.
   specify fitType='local' or 'mean' to avoid this message next time.



In [ ]:
# remove highly correlated data
data_corr <- preProcess(vsd, method = "corr", cutoff = 0.70)
data_trans <- predict(data_corr, vsd)
dim(data_trans)

[1]  534 2407

In [ ]:
data_trans <- merge(data_trans, DGE_metadata['IDH.status'], by = "row.names", sort = FALSE)
dim(data_trans)

[1]  534 2409

In [ ]:
rownames(data_trans) <- data_trans$Row.names
data_trans <- data_trans[,-1]
head(data_trans)

,ENSG00000131095.13,ENSG00000120885.22,ENSG00000113140.11,ENSG00000118271.12,ENSG00000109846.9,ENSG00000130203.10,ENSG00000146648.19,ENSG00000198695.2,ENSG00000171885.18,ENSG00000135821.19,⋯,ENSG00000082482.14,ENSG00000144354.14,ENSG00000171004.18,ENSG00000121900.19,ENSG00000102032.13,ENSG00000146090.16,ENSG00000160211.19,ENSG00000101417.12,ENSG00000183549.10,IDH.status
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
TCGA-CS-4938-01B-11R-1896-07,14.042195,5.603533,11.851646,10.307026,8.844817,10.369810,8.154948,12.698106,13.891425,7.116968,⋯,4.680477,12.435793,7.496905,12.03391,7.733082,5.018913,8.147644,10.471995,10.813665,Mutant
TCGA-CS-4941-01A-01R-1470-07,12.375402,11.513038,9.017664,12.208442,5.814786,10.176636,16.425503,8.823037,13.796068,10.418079,⋯,12.207555,13.166846,6.456163,11.55748,12.501291,3.604534,9.870661,10.413404,10.899173,WT
TCGA-CS-4942-01A-01R-1470-07,12.815996,8.730209,9.701820,10.307026,11.522600,11.543719,10.578318,11.201400,11.485022,10.746416,⋯,11.419911,10.138115,10.318163,11.87830,9.414809,10.896232,8.800690,6.893369,12.178980,Mutant
TCGA-CS-4943-01A-01R-1470-07,7.780189,6.295149,8.623301,5.649594,11.486874,9.156750,12.765411,8.340347,9.134230,7.355507,⋯,13.465718,14.538832,7.245613,10.88667,9.180586,9.928951,9.264067,4.015380,7.228911,Mutant
TCGA-CS-4944-01A-01R-1470-07,14.601860,15.869823,9.761764,14.207815,12.186869,9.160028,8.495910,10.791995,5.405140,9.834508,⋯,5.428684,9.040686,5.559601,12.84305,12.955605,6.123716,9.623670,7.571690,12.710549,Mutant
TCGA-CS-5390-01A-02R-1470-07,6.414462,6.768295,8.117387,8.416037,6.910824,5.125587,13.960940,10.549869,8.420851,7.972475,⋯,9.096286,10.546378,9.795453,7.15523,7.508538,8.622440,8.933746,11.529123,7.259143,Mutant


### **Splitting dataset into training and test datasets**

In [ ]:
# Remove missing values in 'IDH.status'
data_trans <- data_trans[!is.na(data_trans$`IDH.status`), ]
unique(data_trans$`IDH.status`)

# Convert 'IDH.status' to a factor if it's not already
data_trans$`IDH.status` <- as.factor(data_trans$`IDH.status`)

[1] "Mutant" "WT"

In [ ]:
# splitting the dataset in the ratio 80:20
intrain <- createDataPartition(y = data_trans$IDH.status, p = 0.7)[[1]]
train_data <- data_trans[intrain,]
test_data <- data_trans[-intrain,]
dim(train_data)
dim(test_data)

[1]  372 2408

[1]  159 2408

### **Training the Machine Learning Model on the Training dataset**

In [ ]:
# to prevent any bias; done by the cross validation method
control.bias <- trainControl(method = "cv", number = 7)

In [ ]:
#training the model
rf.output <- train(IDH.status~.,
                  data = train_data,
                  method = "ranger",
                  trControl = control.bias,
                  importance = "permutation")

In [ ]:
rf.output$finalModel$prediction.error

[1] 0.02150538

In [ ]:
# predicting on test data
rf.predictions <- predict(rf.output, test_data)

In [ ]:
# Convert rf.predictions and test_data$IDH.status to factors with the same levels
rf.predictions <- as.factor(rf.predictions)
test_data$IDH.status <- as.factor(test_data$IDH.status)

# Set the levels of rf.predictions to be the same as test_data$sample_type
levels(rf.predictions) <- levels(test_data$IDH.status)

In [ ]:
confusionMatrix(rf.predictions, test_data$IDH.status)

Confusion Matrix and Statistics

          Reference
Prediction Mutant  WT
    Mutant    130   2
    WT          0  27
                                          
               Accuracy : 0.9874          
                 95% CI : (0.9553, 0.9985)
    No Information Rate : 0.8176          
    P-Value [Acc > NIR] : 8.23e-12        
                                          
                  Kappa : 0.9567          
                                          
 Mcnemar's Test P-Value : 0.4795          
                                          
            Sensitivity : 1.0000          
            Specificity : 0.9310          
         Pos Pred Value : 0.9848          
         Neg Pred Value : 1.0000          
             Prevalence : 0.8176          
         Detection Rate : 0.8176          
   Detection Prevalence : 0.8302          
      Balanced Accuracy : 0.9655          
                                          
       'Positive' Class : Mutant          
                     